In [0]:

# Code recorded by Artem Remezov

# One of my solutions for 'M5 Forecasting - Accuracy' competition
# (https://www.kaggle.com/c/m5-forecasting-accuracy)
# based on Linear Regression and auto-Arima models




!pip install pmdarima

from math import sqrt
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date, timedelta
import statsmodels.api as sm
import pandas as pd
import numpy as np
import warnings
import itertools
import matplotlib.pyplot as plt
import os
import matplotlib
from typing import Union
from tqdm.auto import tqdm as tqdm
from sklearn.linear_model import LinearRegression
import pmdarima as pm
from math import isnan as math_isnan
import math, decimal
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler



# Receiving a dataframe, calculating correlations,
# appending correlations with absolute value higher than corr_benchmark
# to the dictionary
def correlations_to_dict(input_df, input_dict, corr_benchmark):
    
    
    cols_to_delete = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    labels = list(df_for_corr['id'])
    input_df.drop(cols_to_delete, axis=1, inplace=True)
    input_df = input_df.transpose()
    input_df.columns = labels
    test_corr = input_df.corr()
    
    
    for i, row in test_corr.iterrows():
        for j, column in row.iteritems():
            if not(math_isnan(column)):
                if abs(column) >= corr_benchmark and i != j:
                    #print(i, '...', j, '...', column)
                    if i in input_dict:
                        t = input_dict[i]
                        t += [j]
                        input_dict[i] = t
                    else:
                        input_dict[i] = [j]

                        
# Returns a dataframe with all prices for an item
def get_s_prices(t_item):

    # Get min and max indexes from the sorted dataframe in order to use .loc method and save time
    st_pos = items_and_prices['item'].values.searchsorted(t_item, side='left')
    end_pos = items_and_prices['item'].values.searchsorted(t_item, side='right') - 1
    item_calendar = items_and_prices.loc[st_pos:end_pos, :]

    
    d1 = date(2011,1,29)
    d2 = date(2016,6,19)

    # Creating a list containing all of the dates
    dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]

    df_dates = pd.DataFrame(dd)
    df_dates.columns = ['date']
    df_dates['date'] =  pd.to_datetime(df_dates['date'], format='%Y-%m-%d')
    
    s_prices = df_dates.merge(item_calendar, how='left', on='date')['sell_price']
    
    return s_prices                       
                        
                        
# Credits to: https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/154776                        
def get_moon_phase(d):  # 0=new, 4=full; 4 days/phase
    dec = decimal.Decimal
    diff = d - datetime(2001, 1, 1)
    days = dec(diff.days) + (dec(diff.seconds) / dec(86400))
    lunations = dec("0.20439731") + (days * dec("0.03386319269"))
    phase_index = math.floor((lunations % dec(1) * dec(8)) + dec('0.5'))
    return int(phase_index) & 7                        

# Returns rollings with a given shift for train and test splits
def shifted_df_for_prediction(input_s, t_shift, window_size, n_to_predict, train_days):
 
    input_df2 = pd.DataFrame(input_s)
    input_df2.reset_index(inplace=True)
    input_df2.drop(['index'], axis=1, inplace=True)
    
    #print(input_df2)
    
    window = input_df2.rolling(window=window_size)
    shift_df = pd.concat([window.std(), window.min(), window.max(), window.mean()], axis=1)
    
    st = shift_df.shape[0] - t_shift
    en = st + n_to_predict - 1
    
    return shift_df[-train_days:], shift_df.loc[st:en, :]


# Adds ratio previous/ current Sale
def add_ratio_to_prices(input_s):
    
    input_df = pd.DataFrame(input_s)
    
    input_df.columns = ['sales']
    input_df['shift_1'] = input_df['sales'].shift(1)
    
    # replacing first NaN value with 1
    input_df.loc[0, 'shift_1'] = 1
    input_df['ratio'] = input_df['shift_1']/input_df['sales']
    
    return input_df


# Creating calendar features
def process_calendar():

    # Black Friday (https://www.timeanddate.com/holidays/us/black-friday)
    black_friday_dates = ['2011-11-25', '2012-11-23', '2013-11-29', '2014-11-28', '2015-11-27']

    for item in black_friday_dates:
        s = calendar['date']==item
        calendar.loc[s, 'black_friday'] = 1

    s = calendar['black_friday'].isnull()==True
    calendar.loc[s, 'black_friday'] = 0  




    # Dates of Ramadan
    Ramadan_dates = [['2011-08-01', '2011-08-30'], 
                     ['2012-07-20', '2012-08-18'], 
                     ['2013-07-09', '2013-08-07'], 
                     ['2014-06-29', '2014-07-27'], 
                     ['2015-06-18', '2015-07-17'], 
                     ['2016-06-07', '2016-07-06']]

    for item in Ramadan_dates:
        s = (calendar['date'] >= item[0]) & (calendar['date'] <= item[1])
        calendar.loc[s, 'Ramadan'] = 1

    s = calendar['Ramadan'].isnull()==True
    calendar.loc[s, 'Ramadan'] = 0



    # Processing default fields from the calendar
    for item in ['Religous', 'National', 'Cultural', 'Sporting']:
        s = (calendar['event_type_1']==item)|(calendar['event_type_2']==item)
        calendar.loc[s, item + '_events'] = 1

    for item in ['Religous_events', 'National_events', 'Cultural_events', 'Sporting_events']:   
        s = calendar[item].isnull()==True
        calendar.loc[s, item]=0




    # Weekend
    s = calendar['weekday'].isin(['Saturday', 'Sunday'])
    calendar.loc[s, 'weekend'] = 1
    s = calendar['weekend'].isnull()==True
    calendar.loc[s, 'weekend'] = 0






    # First day of the month
    s = calendar['date'].dt.day == 1
    calendar.loc[s, 'first_day_of_month'] = 1
    s = calendar['first_day_of_month'].isnull()==True
    calendar.loc[s, 'first_day_of_month'] = 0


    # Seasons
    for item1, item2 in zip([[12, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]], 
                             ['winter', 'spring', 'summer', 'fall']):
        s = calendar['date'].dt.month.isin(item1)
        calendar.loc[s, item2] = 1
        s = calendar[item2].isnull()==True
        calendar.loc[s, item2] = 0



    nba_finals_dates = [
        '2011-05-31', '2011-06-02', '2011-06-05', '2011-06-07',
        '2011-06-09', '2011-06-12', '2012-06-12', '2012-06-14', 
        '2012-06-17', '2012-06-19','2012-06-21',  '2013-06-06', 
        '2013-06-09', '2013-06-11', '2013-06-13', '2013-06-16', 
        '2013-06-18', '2013-06-20', '2014-06-05', '2014-06-08', 
        '2014-06-10', '2014-06-12', '2014-06-15', '2015-06-04', 
        '2015-06-07', '2015-06-09', '2015-06-11', '2015-06-14', 
        '2015-06-16', '2016-06-02', '2016-06-05', '2016-06-08', 
        '2016-06-10', '2016-06-13', '2016-06-16', '2016-06-19']


    for item in nba_finals_dates:
        s = calendar['date']==item
        calendar.loc[s, 'nba_final'] = 1

    s = calendar['nba_final'].isnull()==True
    calendar.loc[s, 'nba_final'] = 0    




    # Fourier terms

    calendar['date_todatetime'] = pd.to_datetime(calendar['date'])
    calendar['dayofyear'] = calendar['date_todatetime'].dt.dayofyear


    calendar['sin365'] = np.sin(2 * np.pi * calendar['dayofyear']  / 365.25)
    calendar['cos365'] = np.cos(2 * np.pi * calendar['dayofyear']  / 365.25)
    calendar['sin365_2'] = np.sin(4 * np.pi * calendar['dayofyear']  / 365.25)
    calendar['cos365_2'] = np.cos(4 * np.pi * calendar['dayofyear'] / 365.25)

    
    calendar['week_sin365'] = np.sin(2 * np.pi * calendar['dayofyear']  / 7)
    calendar['week_cos365'] = np.cos(2 * np.pi * calendar['dayofyear']  / 7)
    calendar['week_sin365_2'] = np.sin(4 * np.pi * calendar['dayofyear']  / 7)
    calendar['week_cos365_2'] = np.cos(4 * np.pi * calendar['dayofyear'] / 7)

    calendar['month_sin365'] = np.sin(2 * np.pi * calendar['dayofyear']  / 30)
    calendar['month_cos365'] = np.cos(2 * np.pi * calendar['dayofyear']  / 30)
    calendar['month_sin365_2'] = np.sin(4 * np.pi * calendar['dayofyear']  / 30)
    calendar['month_cos365_2'] = np.cos(4 * np.pi * calendar['dayofyear'] / 30)



    # Moon phases
    calendar['moon'] = calendar.date.apply(get_moon_phase)

    
# Linear Regression + auto Arima (optional) on residuals from linear regression
def model_results_to_dfs(dict_corr, input_df, res_df, exog_s):
    
    b_test = False
    already_predicted = {}
    cnt_used_corr = 0

    


    scaler = MinMaxScaler(feature_range=(0, 1))
    
    st = total - n_train_days
   
    st_pred = total - st + 1
        
    end_pred = st_pred + to_predict - 1
    
    
    
    if b_test:
        print('st: ', st)
        print('st_pred: ', st_pred)
        print('end_pred: ', end_pred)
        
        
    # exog_s = DataFrame
    exog_to_fit = exog_s.loc[0:total-1]
    if b_test:
        print('exog_to_fit.shape: ', exog_to_fit.shape)
    e = exog_to_fit.loc[st:].astype('float64')      
    e = np.asarray(e)
    if b_test:
        print('e.shape: ', e.shape)
    
    
    ep = exog_s.loc[exog_s.shape[0] - to_predict - n_train_days:exog_s.shape[0] - to_predict - 1].astype('float64')
    ep = np.asarray(ep)
    
    
    epf = exog_s.loc[exog_s.shape[0] - to_predict:].astype('float64')
    epf = np.asarray(epf)
    
    
    if b_test:
        print('ep.shape: ', ep.shape)
        print('epf.shape: ', epf.shape)
    
    
    
    print('start:')
    print(datetime.now())
    
    cnt = 0
    
    res_df1 = res_df.copy()
    #res_df2 = res_df.copy()
    
    
    # Iterating over the dataframe rows
    for i, row in input_df.iterrows():
        
        cnt += 1
        tmp = input_df.loc[i, :]
        id = tmp['id']

        s = tmp.drop(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
        lags_train1, lags_test1 = shifted_df_for_prediction(s, 365, 7, to_predict, n_train_days)
        lags_train2, lags_test2 = shifted_df_for_prediction(s, 28, 7, to_predict, n_train_days)
        
        s = np.asarray(s[st:].astype('float64'))
        
        # Applying sell prices if necessary
        if use_prices:

            cp = get_s_prices(id.replace('_evaluation', ''))
            cp.fillna(method='bfill', inplace=True)
            cp = np.asarray(cp).reshape(len(cp),1)

            cp = add_ratio_to_prices(cp)
            
            cp = np.asarray(cp)
            
            scaler = scaler.fit(cp)
            cp = scaler.transform(cp)

            cp_actual = cp[st:total]
            cp_predict = cp[-to_predict:]

     

        if sum(s)==0:
            predictions = [0 for item in range(to_predict)]
        else:
            

            ep2 = np.copy(ep)
            epf2 = np.copy(epf)
            ep2 = np.concatenate([ep, lags_train1], axis=1)
            epf2 = np.concatenate([epf, lags_test1], axis=1)            


            
            ep2 = np.concatenate([ep2, lags_train2], axis=1)
            epf2 = np.concatenate([epf2, lags_test2], axis=1)  
            
            if use_prices:
                ep2 = np.concatenate([ep, cp_actual], axis=1)
                epf2 = np.concatenate([epf, cp_predict], axis=1)            
            
            
            
            if use_correlations: 

                if id in dict_corr:
                    t = dict_corr[id]
                    if not(t is None):
                        for item in t:
                            if item in already_predicted:

                                t1 = already_predicted[item][0]
                                t1 = t1.reshape(len(t1), 1)
                                ep2 = np.concatenate([ep2, t1], axis=1)


                                t2 = already_predicted[item][1]
                                t2 = t2.reshape(len(t2), 1)
                                epf2 = np.concatenate([epf2, t2], axis=1)
                                cnt_used_corr += 1
                            
            linreg = LinearRegression().fit(ep2, s)
            
            
            reg_predicted = linreg.predict(epf2)
            
            if use_Arima:

                predicted_for_actual = linreg.predict(ep2)
            
                resid = s - predicted_for_actual
           
                autoarima = pm.auto_arima(resid[-n_train_days_Arima:], start_p=0, d=0, start_q=0,
                                          max_p=2, max_d=1, max_q=2,
                                          start_P=0, D=0, start_Q=0,
                                          max_P=2, max_D=1, max_Q=2,
                                          stepwise=True, error_action='ignore', seasonal=True, m=7)
           
                results_autoarima = autoarima.fit(resid[-n_train_days_Arima:])
            
                predicted_autoarima = results_autoarima.predict(n_periods=to_predict)
            
            
                predictions = list(reg_predicted + predicted_autoarima)
            else:
                predictions = list(reg_predicted)
                
                
            if use_correlations:
                already_predicted[id] = [s, np.array(predictions)]
                        
            predictions = [0 if item < 0 else item for item in predictions]            
            


        t1 = predictions[:]        
        
        list_out1 = [id] + t1             
        
        series_out1 = pd.Series(list_out1, index=sample_submission.columns) 
                
        res_df1 = res_df1.append(series_out1, ignore_index=True)
        
        
        if cnt % 1000 == 0:
            print('Rows processed: ', str(cnt))
            #print(datetime.now())
            
            
    if use_correlations:  
        print('n of outer correlations used: ', str(cnt_used_corr))
    
    
    print('end:')
    print(datetime.now())
    return res_df1
    

def load_dataframes():
    calendar_path = folder_path + 'calendar.csv'
    sales_train_validation_path = folder_path + 'sales_train_evaluation.csv'
    sample_submission_path = folder_path + 'sample_submission.csv'
    sell_prices_path = folder_path + 'sell_prices.csv'   


    # Uploading files to dataframes
    calendar = pd.read_csv(calendar_path, sep=',')
    sales_train_validation = pd.read_csv(sales_train_validation_path, sep=',')
    sample_submission = pd.read_csv(sample_submission_path, sep=',')
    sell_prices = pd.read_csv(sell_prices_path, sep=',')


    # Converting 'date' to date format
    calendar['date'] =  pd.to_datetime(calendar['date'], format='%Y-%m-%d')

    return calendar, sales_train_validation, sample_submission, sell_prices


### Main ###
folder_path = '/kaggle/input/m5-forecasting-accuracy/'


total = 1941
n_train_days = 273
n_train_days_Arima = 70
to_predict = 28

cutoff_correlation = 0.5


full_run = True
partial_run = False
curr_part = 1


use_correlations = False
use_prices = True
use_Arima = False


if partial_run:    
    corr_by_field = 'state_id'
    if curr_part==1:
        t_field_value='CA'
    elif curr_part==2:
        t_field_value='TX'
    elif curr_part==3:
        t_field_value='WI'
  



# Revise mode in case of a mistake
if full_run and partial_run:
    partial_run = False

if not(full_run) and not(partial_run):
    full_run = True

if n_train_days_Arima > n_train_days:
    n_train_days_Arima = n_train_days


# Loading dataframes
calendar, sales_train_validation, sample_submission, sell_prices = load_dataframes()

# Applying additional features to the calendar
process_calendar()

# if NA for sell price - next valid price is used
if use_prices:
    print('Start processing items_and_prices df: ', datetime.now())
    sell_prices['item'] = sell_prices['item_id'] + '_' + sell_prices['store_id']

    calendar2 = calendar[['date', 'wm_yr_wk']]
    items_and_prices = sell_prices.merge(calendar2, how='inner', on='wm_yr_wk')[['item', 'date', 'sell_price']]
    items_and_prices.sort_values(inplace=True, by='item')
    items_and_prices.reset_index(inplace=True, drop=True)
    print('End processing items_and_prices df: ', datetime.now())




# Appending inner correlations to dictionary
# Splitting into 3 groups (=states: California, Texas, Wisconsin) because a run on the full dataframe may
# last longer than Kaggle session lifetime

dict_corr = {}

if use_correlations:
    
    print('Collecting correlations, start:')
    print(datetime.now())
    
    s = sales_train_validation[corr_by_field]==t_field_value
    df_for_corr = sales_train_validation.loc[s, :]
    correlations_to_dict(df_for_corr, dict_corr, cutoff_correlation)   
    
    print('Collecting correlations, end:')
    print(datetime.now())




# Creating submission file template
res_df = pd.DataFrame(data=None, columns=sample_submission.columns, index=sample_submission.index) 
res_df.drop(res_df.index[0:res_df.shape[0]], axis=0, inplace=True)




if partial_run:  
    if curr_part==1:
        t1 = sales_train_validation.loc[0:12195, :]
        #t1 = sales_train_validation.loc[0:1, :]
        out_file = 'submission1.csv'
    elif curr_part==2:
        t1 = sales_train_validation.loc[12196:21342, :]
        out_file = 'submission2.csv'
    elif curr_part==3:
        t1 = sales_train_validation.loc[21343:30489, :]
        out_file = 'submission3.csv'

        
if full_run:
    out_file = 'submission.csv'
    t1 = sales_train_validation.loc[:, :]

    

    

# Exogenous variables from the calendar to include into the model
list_exog_to_include = ['sin365', 'cos365', 'sin365_2', 'cos365_2',
                        'week_sin365', 'week_cos365', 'week_sin365_2', 'week_cos365_2', 
                        'month_sin365', 'month_cos365', 'month_sin365_2', 'month_cos365_2',
                        'snap_CA', 'snap_TX', 'snap_WI',
                        'black_friday', 'Ramadan', 'Religous_events', 'National_events', 
                        'Cultural_events', 'Sporting_events', 'weekend', 'first_day_of_month',
                        'winter', 'spring', 'summer', 'fall', 'moon', 'nba_final']

exog_s_to_model = calendar[list_exog_to_include]


# Creating a dataframe with validation data
i = ['d_' + str(item) for item in range(1914, 1942)]

sub_validation = sales_train_validation[i]

i = ['F' + str(item) for item in range(1, 29)]

sub_validation.columns = i

sub_validation_ids = sales_train_validation[['id']]

sub_validation = pd.concat([sub_validation_ids, sub_validation], axis=1)

sub_validation['id'] = sub_validation['id'].str.replace('evaluation', 'validation')
        


res_df1 = model_results_to_dfs(dict_corr, t1, res_df, exog_s_to_model)

submission_df = pd.concat([res_df1, sub_validation], ignore_index=True)


# Saving the result to .csv
submission_df.to_csv(out_file, sep=',', index=False)

    


